In [11]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from typing import List, TypedDict, Literal, Annotated
import operator

# --- SETUP ---
load_dotenv()


google_api_key = os.getenv("GEMNI_API_KEY")

if not google_api_key:
    raise ValueError("GEMNI_API_KEY not found! Please set it in your .env file.")

print("API key loaded")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", temperature=0.3, api_key=google_api_key
)


API key loaded


In [12]:
class HybridState(TypedDict):
    # Plan-Execute Fields
    input: str
    plan: List[str]
    current_step: int
    results: Annotated[List[str], operator.add] # Appends new results to the list
    
    # Reflection Fields
    draft: str
    critique: str
    reflection_iterations: int
    
    # Final Output
    final_output: str

# Constants
MAX_REFLECTION_ITERATIONS = 2

In [13]:

# 1. Planner: Breaks the task into steps
def planner_node(state: HybridState) -> dict:
    print(f"\n{'='*20} PLANNER {'='*20}")
    prompt = f"""You are a planner. Create a step-by-step plan for this task:
Task: {state['input']}

Return ONLY a numbered list of steps (maximum 3 steps).
Do not include any intro or outro text."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Simple parsing to get the lines that look like steps
    steps = [line.strip() for line in response.content.split('\n') 
             if line.strip() and (line[0].isdigit() or line.startswith('-'))]
    
    print(" Plan Created:")
    for step in steps:
        print(f"   {step}")
        
    return {
        "plan": steps, 
        "current_step": 0, 
        "results": [], 
        "reflection_iterations": 0
    }

# 2. Executor: Performs one step of the plan
def executor_node(state: HybridState) -> dict:
    plan = state["plan"]
    step_idx = state["current_step"]
    
    if step_idx >= len(plan):
        return {} # Safety valve
    
    current_task = plan[step_idx]
    print(f"\n Executing Step {step_idx + 1}: {current_task}")
    
    # Include context from previous steps if available
    context = "\n".join(state["results"])
    prompt = f"""Execute this specific step of the plan: {current_task}
    
Context from previous steps:
{context}

Provide the result for this step only."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    result = f"Step {step_idx + 1} Result: {response.content}"
    
    return {
        "results": [result],
        "current_step": step_idx + 1
    }
def generator_node(state: HybridState) -> dict:
    print(f"\n{'='*20} GENERATOR {'='*20}")
    
    all_research = "\n\n".join(state["results"])
    
    prompt = f"""Synthesize the following research into a cohesive response for the task.
    
Original Task: {state['input']}

Research Materials:
{all_research}

Write a complete initial draft."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    print("✍️ Initial Draft Generated")
    return {"draft": response.content}

# 4. Critic: Evaluates the draft
def critic_node(state: HybridState) -> dict:
    print(f"\n🔍 Critiquing Draft (Iteration {state['reflection_iterations'] + 1})...")
    
    prompt = f"""Critique this draft based on the original task.
    
Task: {state['input']}
Draft: {state['draft']}

Provide constructive feedback on how to improve it.
If the draft is excellent and meets all requirements, explicitly say "APPROVED".
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "critique": response.content,
        "reflection_iterations": state["reflection_iterations"] + 1
    }

# 5. Refiner: Improves draft based on critique
def refiner_node(state: HybridState) -> dict:
    print(f"🛠️ Refining Draft...")
    
    prompt = f"""Rewrite and improve the draft based on the critique.
    
Task: {state['input']}
Current Draft: {state['draft']}
Critique: {state['critique']}

Return the improved draft only."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content}

In [14]:
def finalizer_node(state: HybridState) -> dict:
    print(f"\n{'='*20} PROCESS COMPLETE {'='*20}")
    return {"final_output": state["draft"]}

# --- ROUTING LOGIC ---

def should_continue_execution(state: HybridState) -> Literal["executor", "generator"]:
    # Loop until all plan steps are finished
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"

def should_refine(state: HybridState) -> Literal["refiner", "finalizer"]:
    # Stop if approved or max iterations reached
    if "APPROVED" in state["critique"].upper():
        print("    Draft Approved by Critic")
        return "finalizer"
    
    if state["reflection_iterations"] >= MAX_REFLECTION_ITERATIONS:
        print("   ⚠️ Max reflection iterations reached")
        return "finalizer"
    
    return "refiner"


In [15]:

builder = StateGraph(HybridState)

# Add Nodes
builder.add_node("planner", planner_node)
builder.add_node("executor", executor_node)
builder.add_node("generator", generator_node)
builder.add_node("critic", critic_node)
builder.add_node("refiner", refiner_node)
builder.add_node("finalizer", finalizer_node)

# Define Flow
builder.add_edge(START, "planner")
builder.add_edge("planner", "executor")

# Execution Loop
builder.add_conditional_edges(
    "executor",
    should_continue_execution,
    {"executor": "executor", "generator": "generator"}
)

builder.add_edge("generator", "critic")

# Reflection Loop
builder.add_conditional_edges(
    "critic",
    should_refine,
    {"refiner": "refiner", "finalizer": "finalizer"}
)

builder.add_edge("refiner", "critic") # Loop back to see if critique improved
builder.add_edge("finalizer", END)

# Compile
hybrid_agent = builder.compile()

In [19]:

print(f"🚀 STARTING HYBRID AGENT TEST\n")

# Use a vague task to ensure the agent has to plan AND refine
query = "Research the benefits of artificial intelligence, create a summary, and make it beginner-friendly"

try:
    result = hybrid_agent.invoke({
        "input": query,
        "plan": [],
        "current_step": 0,
        "results": [],
        "draft": "",
        "critique": "",
        "reflection_iterations": 0,
        "final_output": ""
    })

    print(f"\n\n{'='*80}")
    print("📊 FINAL HYBRID OUTPUT")
    print(f"{'='*80}")
    print(result["final_output"])
    print(f"{'='*80}")

except Exception as e:
    print(f"\n Error during execution: {e}")
    print("If this is a ConnectError, please check your internet connection or disable your VPN.")

🚀 STARTING HYBRID AGENT TEST


==================== PLANNER ====================
 Plan Created:
   1.  **Research and Identify Key Benefits:** Conduct broad research on the advantages of AI across various sectors (e.g., healthcare, business, education, daily life), focusing on tangible improvements and positive impacts.
   2.  **Synthesize and Simplify Information:** Consolidate the research findings into core themes and benefits, then rephrase complex AI concepts and their advantages using simple language, analogies, and relatable examples suitable for someone with no prior AI knowledge.
   3.  **Structure and Format for Beginners:** Organize the simplified information into a clear, concise, and visually appealing summary, using headings, bullet points, and minimal jargon to ensure easy comprehension and engagement for a beginner audience.

 Executing Step 1: 1.  **Research and Identify Key Benefits:** Conduct broad research on the advantages of AI across various sectors (e.g., health